## A simplified unit commitment problem

### Part A

The unit commitment problem consists of determining a **cost minimizing** power generation schedule, so as to satisfy a forecasted power demand (load) over a given number of periods.

Consider the following simplified version of the problem: The scheduling horizon includes eight
periods ($T=8$), corresponding to a three-hour resolution for a full day. The forecasted load for
each period is reported in the table below:

time period $t$|1|2|3|4|5|6|7|8
---------------|-|-|-|-|-|-|-|-
load forecast $d_t$ (in MW)|1.3|3.2|10.6|5.7|15.9|6.8|18.1|8.9

There are three available generators which can be scheduled for satisfying the load. For each generator, there is variable cost $c_i$, which is the cost incurred for each MW produced by generator $i$, and a cost $b_i$, which is incurred for each period in which $i$ is operated (ON). Parameters $p^{min}_{i}$
and $p^{max}_{i}$ are the minimum and maximum production levels of $i$ per period, respectively. The values of all parameters characterizing each generator are:

generator $i$|1|2|3
-------------|-|-|-
$c_i$|7|10|6.5
$b_i$|30|40|60
$p^{min}_{i}$|1|3.2|4.1
$p^{max}_{i}$|6.5|10|30

Provide a mixed integer linear programming formulation of this problem, and solve it with mip.

### Sets:
*   $I$: set of power generators ($|I| = m$)
*   $N$: set of time periods in the scheduling horizon ($|N| = T$)

### Parameters:
*   $d_t$: forecasted demand for period $t \in N$
*   $c_i$: cost per MW produced by generator $i \in I$
*   $b_i$: cost incurred for each period generator i is operated (ON)
*   $p^{min}_i$: minimum production level of generator i per time period (if active)
*   $p^{max}_i$: maximum production level of generator i per time period

### Variables:
*   $x_{it} \ge 0$: amount of power (in MW) produced by generator i during time period t
*   $y_{it} \in \{0, 1\}$: 1 if generator i produces power during time period t (is active as of the beginning of t), 0 otherwise

### Part A Formulation

$$
\begin{array}{lll}
\min & \sum_{i \in I} {\sum_{t \in N} (c_i x_{it} + b_i y_{it})} \\
\textrm{s.t.} & \sum_{i \in I} x_{it} \ge d_t & \forall t \in N \\
              & x_{it} \ge p^{min}_i y_{it} & \forall i \in I, \forall t \in N \\
              & x_{it} \le p^{max}_i y_{it} & \forall i \in I, \forall t \in N \\
              & y_{it} \in \{0,1\} & \forall i \in I, \forall t \in N \\
              & x_{it} \ge 0 & \forall i \in I, \forall t \in N
\end{array}
$$

### Part A Formulation

$$
\begin{array}{lll}
\min & TODO \\
\textrm{s.t.} & TODO
\end{array}
$$

In [2]:
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 5.2 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.15.1
    Uninstalling cffi-1.15.1:
      Successfully uninstalled cffi-1.15.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 5.2 MB/s 


In [7]:
import mip

T = 8
N = range(8)

m = 3
I = range(m)

d = [1.3, 3.2, 10.6, 5.7, 15.9, 6.8, 18.1, 8.9]
c = [7, 10, 6.5]
b = [30, 40, 60]
p_min = [1, 3.2, 4.1]
p_max = [6.5, 10, 30]

m = mip.Model()

x = [[m.add_var() for t in N] for i in I]
y = [[m.add_var(var_type=mip.BINARY) for t in N] for i in I]

for t in N:
  m.add_constr(mip.xsum(x[i][t] for i in I) >= d[t])

for i in I:
  for t in N:
    m.add_constr(x[i][t] <= p_max[i] * y[i][t])
    m.add_constr(x[i][t] >= p_min[i] * y[i][t])

m.objective = mip.minimize(mip.xsum(c[i] * x[i][t] + b[i] * y[i][t] for i in I for t in N))

m.optimize()
print(m.objective_value)

853.35
---------
[1.3, 3.2, 0.0, 5.7, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 10.6, 0.0, 15.9, 6.8, 18.1, 8.9]
---------
[1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]


### Part B

Each generator has a minimum up time and a minimum down time. Therefore, once turned on, generator $i$ must stay on for at least $L_o = 3$ time periods. Once turned off, generator $i$ must stay off for at least $L_f = 2$ time periods.

Modify the formulation you proposed in Part A in a manner that accommodates
these constraints, and solve it using mip.

Assumptions: Since we do not know the state of the generators (whether on/off) prior to the beginning of the given scheduling horizon $T$, we are allowed to turn them on in period $1$. Furthermore, we assume generators will be turned off once the scheduling horizon has terminated; therefore we make sure any generator that was off at $t−1$ cannot be turned on at $t$ if the number of time periods from $t$ until the end of the scheduling horizon is less than $L_o$.

Additional parameters:
*   $L_o$: minimum up time (in number of periods)
*   $L_f$: minimum down time (in number of periods)

Additional constraints:

$$
\begin{array}{lll}
& \sum_{τ \in \{1,...,L_o\}} y_{iτ} \ge L_o y_{i1} & \forall i \in I \\
& \sum_{τ \in \{0,...,L_o - 1\}} y_{i(τ + t)} \ge L_o(y_{it} - y_{i(t-1)}) & \forall i \in I, t \in \{2,...,T - L_o + 1\} \\
& \sum_{τ \in \{T - L_o + 1,...,T\}} y_{iτ} \le L_o y_{i(T - L_o + 1)} & \forall i \in I \\
& \sum_{τ \in \{0,...,L_f - 1\}} (1 - y_{i(τ+t)}) \ge L_f (y_{i(t-1)} - y_{it}) & \forall i \in I, \forall t \in \{2,...,T - L_f + 1\}
\end{array}
$$

In [9]:
Lo = 3
Lf = 2

for i in I:
  m.add_constr(mip.xsum(y[i][k] for k in range(Lo)) >= Lo * y[i][0])

for i in I:
  for t in range(1, T - Lo):
    m.add_constr(mip.xsum(y[i][k + t] for k in range(Lo)) >= Lo * (y[i][t] - y[i][t - 1]))

for i in I:
  m.add_constr(mip.xsum(y[i][k] for k in range(T - Lo, T)) <= Lo * y[i][T - Lo])

for i in I:
  for t in range(1, T - Lf):
    m.add_constr(mip.xsum(1 - y[i][k + t] for k in range(Lf)) >= Lf * (y[i][t - i] - y[i][t]))

m.optimize()
print(m.objective_value)

959.25


### Part C

To avoid damaging the generator its power production cannot change by more than a certain amount from one period to another. Let $\delta^u_i$ be the maximum ramp up rate for generator $i$, i.e., for generator $i$ the production level at $t + 1$ minus the production level at $t$ cannot exceed $\delta^u_i$. Let $\delta^d_i$ be the maximum ramp down rate for generator $i$, i.e., for generator $i$ the production level at $t$ minus the production level at $t + 1$ cannot exceed $\delta^d_i$. Consider values:

generator $i$|1|2|3
------------|-|-|-
$\delta^d_i$|3|5|10
$\delta^u_i$|3|5|10

Modify the formulation you proposed in Part A in a manner that accommodates
these constraints, and solve it using mip.

Assumption: Generators have been off before the considered scheduling horizon and their initial production level is equal to zero.

Additional parameters:
*   $\delta^u_i$: ramp up rate for generator i (maximum allowed increase in the production level between two consecutive time periods)
*   $\delta^d_i$: ramp down rate for generator i (maximum allowed decrease in the production level between two consecutive time periods)

Additional constraints:

$$
\begin{array}{lll}
& x_{i1} \le \delta^u_i & \forall i \in I \\
& x_{it} - x_{i(t-1)} \le \delta^u_i & \forall i \in I, \forall t \in \{2,...,T\} \\
& x_{i(t-1)} - x_{it} \le \delta^d_i & \forall i \in I, \forall t \in \{2,...,T\}
\end{array}
$$

In [11]:
delta_u = [3, 5, 10]
delta_d = [3, 5, 10]

for i in I:
  m.add_constr(x[i][0] <= delta_u[i])

  for t in range(1, T):
    m.add_constr(x[i][t] - x[i][t - 1] <= delta_u[i])
    m.add_constr(x[i][t - 1] - x[i][t] <= delta_d[i])

m.optimize()
print(m.objective_value)

972.0500000000001
